# Simulation: Gamma method-of-moments estimators

This notebook simulates samples from a Gamma distribution and checks whether the **method of moments (MoM)** estimators for the parameters are biased.

## Parameterization used here
We use the **shape–scale** parameterization:

- $X \sim \mathrm{Gamma}(k,\,\theta)$
- Mean: $\mathbb{E}[X] = k\theta$
- Variance: $\mathrm{Var}(X) = k\theta^2$

MoM equates sample mean and (population) sample variance to these moments.


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

np.random.seed(17)

def mean_error(estimates, actual):
    """Mean error = E[estimate - actual] estimated via simulation."""
    est = np.asarray(estimates)
    return float(np.mean(est - actual))

def rmse(estimates, actual):
    """Root mean squared error estimated via simulation."""
    est = np.asarray(estimates)
    return float(np.sqrt(np.mean((est - actual)**2)))


## Method-of-moments estimators

Let $\bar X$ be the sample mean and $S^2$ be the sample variance.

For $\mathrm{Gamma}(k,\theta)$ (shape–scale):

\[
\bar X \approx k\theta, \qquad S^2 \approx k\theta^2
\]

Solving gives MoM estimators:

\[
\hat k = \frac{\bar X^2}{S^2}, \qquad \hat\theta = \frac{S^2}{\bar X}
\]

Important detail: MoM is naturally paired with the **population** version of sample variance (ddof=0):
\[
S^2 = \frac{1}{n}\sum_{i=1}^n (X_i-\bar X)^2
\]
But we’ll also compute the variant using the unbiased variance (ddof=1) to see how it changes bias.


In [ ]:
def mom_gamma_shape_scale(xs, ddof=0):
    """MoM estimators (k, theta) for Gamma(shape=k, scale=theta).

    ddof=0 uses variance with 1/n; ddof=1 uses variance with 1/(n-1).
    """
    xbar = float(np.mean(xs))
    s2 = float(np.var(xs, ddof=ddof))
    if xbar <= 0 or s2 <= 0:
        return np.nan, np.nan
    k_hat = xbar**2 / s2
    theta_hat = s2 / xbar
    return k_hat, theta_hat

def simulate_bias(k_true=3.0, theta_true=2.0, n=20, m=20000, ddof=0):
    k_hats = []
    theta_hats = []
    for _ in range(m):
        xs = np.random.gamma(shape=k_true, scale=theta_true, size=n)
        k_hat, theta_hat = mom_gamma_shape_scale(xs, ddof=ddof)
        k_hats.append(k_hat)
        theta_hats.append(theta_hat)
    k_hats = np.asarray(k_hats)
    theta_hats = np.asarray(theta_hats)
    return {
        'k_mean': float(np.mean(k_hats)),
        'theta_mean': float(np.mean(theta_hats)),
        'k_mean_error': mean_error(k_hats, k_true),
        'theta_mean_error': mean_error(theta_hats, theta_true),
        'k_rmse': rmse(k_hats, k_true),
        'theta_rmse': rmse(theta_hats, theta_true),
        'k_hats': k_hats,
        'theta_hats': theta_hats,
    }


## Run one experiment

Change $(k,\theta)$, sample size $n$, and number of simulations $m$ if you want.


In [ ]:
k_true = 3.0
theta_true = 2.0
n = 20
m = 20000

res_ddof0 = simulate_bias(k_true, theta_true, n=n, m=m, ddof=0)
res_ddof1 = simulate_bias(k_true, theta_true, n=n, m=m, ddof=1)

print('True (k, theta) =', (k_true, theta_true))
print('n =', n, 'm =', m)

print('\nMoM using var ddof=0 (1/n):')
print('  E[k_hat]      =', round(res_ddof0['k_mean'], 4), ' mean error =', round(res_ddof0['k_mean_error'], 4))
print('  E[theta_hat]  =', round(res_ddof0['theta_mean'], 4), ' mean error =', round(res_ddof0['theta_mean_error'], 4))

print('\nMoM using var ddof=1 (1/(n-1)):',)
print('  E[k_hat]      =', round(res_ddof1['k_mean'], 4), ' mean error =', round(res_ddof1['k_mean_error'], 4))
print('  E[theta_hat]  =', round(res_ddof1['theta_mean'], 4), ' mean error =', round(res_ddof1['theta_mean_error'], 4))


## Visualize sampling distributions


In [ ]:
def hist_with_truth(data, truth, title):
    plt.figure(figsize=(7,4))
    plt.hist(data, bins=60)
    plt.axvline(truth, linewidth=2)
    plt.title(title)
    plt.xlabel('estimate')
    plt.ylabel('count')
    plt.show()

hist_with_truth(res_ddof0['k_hats'], k_true, 'MoM $\\hat{k}$ (ddof=0)')
hist_with_truth(res_ddof0['theta_hats'], theta_true, 'MoM $\\hat{\\theta}$ (ddof=0)')


## Bias vs sample size
This shows how bias changes as $n$ increases.


In [ ]:
def bias_vs_n(k_true=3.0, theta_true=2.0, ns=(5, 10, 20, 50, 100), m=20000, ddof=0):
    out = []
    for n in ns:
        r = simulate_bias(k_true, theta_true, n=n, m=m, ddof=ddof)
        out.append((n, r['k_mean_error'], r['theta_mean_error']))
    return out

ns = (5, 10, 20, 50, 100)
out0 = bias_vs_n(k_true, theta_true, ns=ns, m=20000, ddof=0)
out1 = bias_vs_n(k_true, theta_true, ns=ns, m=20000, ddof=1)

print('ddof=0: (n, bias(k_hat), bias(theta_hat))')
for row in out0:
    print(row)

print('\nddof=1: (n, bias(k_hat), bias(theta_hat))')
for row in out1:
    print(row)


## Conclusion

- In finite samples, the MoM estimators for $k$ and $\theta$ are typically **biased** (mean error not equal to 0).
- The bias usually shrinks as $n$ increases.
- Using ddof=0 vs ddof=1 changes the estimator definition and can change bias.
